# Eigenface Gesichtserkennung
* Autor: Prof. Dr. Johannes Maucher
* Datum: 27.11.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.ipynb)

# Einführung
## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __Gesichtserkennung:__ mit der Eigenface Methode. 
* __Principal Component Analysis__
* __Bildverarbeitung__ mit Python.

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

Die Gesichtserkennung kann mit unterschiedlichen Ansätzen realisiert werden. In diesem Versuch wird ausschließlich der _Eigenface_-Ansatz vorgestellt. Dieser Ansatz basiert auf der _Principal Component Analysis (PCA)_ und wurde erstmals in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) vorgestellt. Die Eigenface-Methode weist eine gute Performance im Fall biometrisch aufgenommener Gesichtsbilder auf.

### Das Prinzip der Eigenface Gesichtserkennung

Bilder mit $C$ Pixeln in der Breite und $R$ Pixeln in der Höhe können als $R \times C$ Matrizen abgespeichert werden. Handelt es sich um ein Schwarz-Weiß- oder Graustufen-Bild, dann wird pro Bild nur eine derartige Matrix benötigt. Der Eintrag in der i.ten Zeile und j.ten Spalte dieser Matrix definiert den Grauwert des entsprechenden Pixels. In Farbbildern werden je nach benutztem Farbraum mehrere  Matrizen pro Bild benötigt, wobei jede Matrix einen Farbkanal des Bildes repräsentiert. Für ein RGB-Bild werden z.B. 3 Matrizen für die Farbkanäle Rot, Grün und Blau benötigt. \\

Im Folgenden wird von quadratischen Graubildern mit $N \times N$ Pixeln ausgegangen. Wird jedes Pixel als ein Merkmal betrachtet, dann existieren insgesamt $N^2$ Merkmale, das Bild kann auch als ein Punkt im $N^2$-dimensionalen Raum betrachtet werden. Bilder der Auflösung $256 \times 256$ müßten also im $65536$-dimensionalen Raum beschrieben werden. Entsprechend komplex wäre die notwendige Verarbeitung. Ist jedoch bekannt, dass in einer Menge von Bildern jeweils ein gleichartiges Objekt abgebildet ist, z.B. wenn alle Bilder ausschließlich je ein Gesicht enthalten, dann existieren große Abhängigkeiten zwischen diesen Bildern. Geometrisch ausgedrückt bedeutet dies, dass die Punkte, welche die Menge der gleichartigen Bilder beschreiben, nicht gleichmäßig über den $N^2$-dimensionalen Raum verteilt sind, sondern in einen relativ kleinen Unterraum mit $K<<N^2$ Dimensionen nahezu vollständig beschrieben werden können. Jede dieser $K$ Dimensionen beschreibt ein für die Kategorie (z.B. Gesichtsbilder) relevantes Merkmal. Im Fall der Gesichtserkennung werden die relevanten Merkmale auch als Eigenfaces bezeichnet. Jedes Eigenface kann als Bild dargestellt werden, welches ein bestimmtes Gesichtsmerkmal besonders hervorhebt. Jedes individuelle Bild der Kategorie (d.h. jedes Gesicht) kann dann als Linearkombination der $K$ relevanten Merkmale (der $K$ Eigenfaces) beschrieben werden.\\

Das Problem besteht nun zunächst darin, aus einer Menge von Bildern der gleichen Kategorie die relevanten Merkmale zu finden. Dieses Problem wird durch die Principal Component Analysis (PCA) gelöst. Die PCA, findet in einer Menge von Bildern der gleichen Kategorie die Hauptachsen, also die Richtungen im $N^2$-dimensionalen Raum, entlang derer die Varianz zwischen den gegebenen Bildern am stärksten ist. Der $N^2$-dimensionale Pixelraum wird dann in einen Raum, der durch die gefundenen Hauptachsen aufgespannt wird, transformiert. In diesem in der Anzahl der Dimensionen stark reduzierten Raum wird dann die Bilderkennung durchgeführt. Der hier skizzierte Ansatz der Eigenfaces für die Gesichtserkennung wurde erstmalig in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) beschrieben.

### Genereller Ablauf

Die Gesichtserkennung besteht aus 2 Phasen. In der Trainingsphase werden die Gesichtsbilder der zu erkennenden Personen eingelesen und für diese mit der PCA der Eigenface-Raum berechnet. In der Erkennungsphase wird ein neu aufgenommenes Bild in den Eigenface-Raum transformiert und dort dem naheliegendsten Bild aus der Trainingsmenge zugeordnet.

#### Trainingsphase

1. Lese Gesichtsbilder der Personen, die erkannt werden sollen ein. Die Menge dieser Bilder definiert das Trainingsset
2. Berechne mit der PCA den Eigenface-Raum. Dabei werden nur die K Dimensionen, welche zu den Eigenvektoren mit den größten Eigenwerten gehören ausgewählt. Die zu den K Dimensionen (Eigenvektoren) gehörenden Bilder sind die Eigenfaces.
3. Transformiere jedes Bild der Trainingsmenge in den Eigenface-Raum und erhalte so die entsprechende Repräsentation des Bildes als Punkt im Eigenface-Raum.

#### Erkennungsphase

1. Transformiere das zu erkennende Bild in den Eigenface-Raum und berechne dort die Koordinaten des Bildes hinsichtlich aller K-Dimensionen (Eigenfaces)
2. Bestimme ob das zu erkennende Bild überhaupt eine Gesicht darstellt
3. Bestimme ob das Gesicht zu einer bekannten Person, deren Bild in der Trainingsmenge enthalten ist, gehört.

#### Update (optional)

Füge das erkannte Bild zur Menge der Trainingsbilder hinzu und führe die Schritte der Trainingsphase durch.

### Bestimmung der Eigenfaces
<a id='theoryEig'></a>
Es werden zunächst $M$ Gesichtsbilder der zu erkennenden Personen eingelesen (von jeder zu erkennenden Personen möglichst mehrere Bilder). Es wird davon ausgegangen, dass jedes der Bilder $C$ Pixel breit und $R$ Pixel hoch ist. Das Bild kann dann als $R \times C$ Matrix dargestellt werden. Im Fall eines Graustufenbildes repräsentieren die Pixelwerte den entsprechenden Grauwert. Nach dem Einlesen werden die Bildmatrizen als eindimensionale Vektoren dargestellt. Für diese Umformung werden die Zeilen jeder Matrix von oben nach unten ausgelesen und hintereinander gereiht. Jedes Bild wird dann durch einen Vektor der Länge $Z=R \cdot C$ repräsentiert und kann als Punkt im Z-dimensionalen Raum dargestellt werden. Die M Bildvektoren werden im folgenden mit $$\Gamma _1, \Gamma_2, \ldots, \Gamma_M$$ bezeichnet.

Im nächsten Schritt wird das Durchschnittsbild berechnet
$$
\overline{\Gamma}=\frac{1}{M}\sum_{i=1}^{M}{\Gamma_{i}}
$$

Dieses Durchschnittsbild wird von allen Bildern $\Gamma_i$ abgezogen. Die Menge der so gewonnenen Bildrepräsentationen
$$
\Phi_i=\Gamma_i - \overline{\Gamma}
$$

ist dann mittelwertsfrei. Die Menge  $\Phi_1, \Phi_2, \ldots, \Phi_M$ wird dann einer Principal Component Analysis (PCA) (siehe auch [J. Maucher; Feature Selection and Extraction](https://docs.google.com/document/d/13cc9RGeIvsV5JsC-MsymJ6aaJs_ZsUKEwqaOOO8IL8Q/edit?usp=sharing)) unterzogen. Hierzu werden die mittelwertfreien Bildrepräsentationen $\Phi_i$ in die Spalten einer Matrix geschrieben. Diese Matrix wird im Folgenden mit $X$ bezeichnet. Unter der Annahme, dass die $\Phi_i$ bereits als Spaltenvektoren vorliegen, ist die Matrix $X$ definiert als:

$$
X=\left[ \Phi_1, \Phi_2, \ldots, \Phi_M \right].
$$

Die entsprechende Kovarianzmatrix ist dann
$$
CV=X \cdot X^T.
$$

Für die PCA müssten als nächstes eigentlich die Eigenvektoren und Eigenwerte der Kovarianzmatrix $CV$ berechnet werden. Für den vorliegenden Fall kann allerdings die hierfür notwendige Berechnung aus Komplexitätsgründen nicht realisiert werden. Man beachte dass die Matrix $CV$  $Z$ Spalten und $Z$ Zeilen enthält ($Z$ ist die Anzahl der Pixel in einem Bild) und für diese $Z$ Eigenvektoren und Eigenwerte existieren. Wie in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) beschrieben, existieren im Fall, dass die Anzahl der Bilder $M$ wesentlich kleiner als die Anzahl der Pixel $Z$ ist, nur $M-1$ __relevante Eigenvektoren__, die Eigenwerte aller anderen Eigenvektoren liegen nahe bei Null. Der in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) beschriebene Ansatz geht nun von der $M \times M$ Matrix

$$
X^T \cdot X
$$

aus, für welche die Eigenvektoren und Eigenwerte für eine moderate Bildanzahl $M$ gut berechnet werden können. Per Definition gilt für die Eigenvektoren $\mathbf{v}_i$ und Eigenwerte $\mu_i$ dieser Matrix:

$$
X^T \cdot X \cdot \mathbf{v}_i = \mu_i \mathbf{v}_i .
$$

Werden beide Seiten dieser Matrix linksseitig mit der Matrix $X$ multipliziert,

$$
X \cdot X^T \cdot X \cdot \mathbf{v}_i = \mu_i X \mathbf{v}_i,
$$
dann ist daraus zu erkennen, dass die $M$ Vektoren
$$
\mathbf{u}_i=X \mathbf{v}_i
$$

die Eigenvektoren der Matrix $$CV=X \cdot X^T$$ sind. D.h. es können zunächst die M Eigenvektoren der relativ kleinen Matrix $X^T \cdot X$ bestimmt und aus diesen durch eine einfache Multiplikation mit der Matrix $X$ die relevanten Eigenvektoren der Matrix $CV$ berechnet werden. Da die Matrix $X$ die $M$ Bildrepräsentationen  $\Phi_i$ als Spalten enthält, können die gesuchten Eigenvektoren auch als Linearkombination der $M$ Bilder der Trainingsmenge beschrieben werden:

$$
\mathbf{u}_i=\sum_{k=1}^{M}{v_{i,k}\Phi_k}
$$

wobei mit $v_{i,k}$ die $k.$te Komponente des Vektors $\mathbf{v}_i$ bezeichnet wird. Die Eigenvektoren $\mathbf{u}_i$ werden auch Eigenfaces genannt. Per Definition sind die Eigenvektoren paarweise orthogonal. Jeder Eigenvektor ist ein Spaltenvektor mit $Z$ (=Anzahl der Pixel) Komponenten.

Die $M$ Eigenvektoren werden dann entsprechend der Größe der zugehörigen Eigenwerte $\mu_i$ geordnet. Für die weiteren Schritte kann zum Zwecke einer weiteren Komplexitätsreduktion eine Untermenge der $K$ relevantesten Eigenvektoren benutzt werden (also der $K$ Eigenvektoren mit den höchsten Eigenwerten). Beispielsweise ist in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) für die Erkennung von $M=16$ Personen und eine Auflösung von $256 \times 256$ Pixel meist $K=7$ Eigenvektoren für eine gute Erkennung ausgereicht.

### Gesichtserkennung im Eigenspace
<a id='theoryRec'></a>

Die $K$ ausgewählten Eigenvektoren $\mathbf{u}_1,\mathbf{u}_2,\ldots \mathbf{u}_K$ spannen einen $K-$dimensionalen Raum, den sogenannten Eigenspace auf. Die $K$ Vektoren repräsentieren die $K$ Merkmale hinsichtlich derer die Bilder der Trainingsdatenmenge am stärksten variieren.

Für die Bilderkennung wird jetzt jedes Bild, also sowohl die Bilder aus der Trainingsmenge als auch die zu erkennenden Bilder, in den Eigenspace transformiert. Jedes Bild stellt einen Punkt im Eigenspace dar. Für die Erkennung kann einfach die Distanz des zu erkennenden Bildes zu allen Bildern der Trainingsmenge berechnet werden. Das zu erkennende Bild wird der Person (Bildklasse) zugeordnet, deren Punkt im Eigenspace dem Punkt des zu erkennenden Bildes am nächsten liegt.

Die $K$ Komponenten eines Trainingsbildes werden berechnet, indem das Bild auf den jeweiligen Eigenvektor projiziert wird. Demnach ist die $k.$te Komponente des $i.$ten Trainingsbildes  $\Phi_i$:
$$
\omega_{k,i}=\mathbf{u}_k^T \Phi_i
$$

Der dem Bild $\Phi_i$ entsprechende Punkt im Eigenspace ist dann
$$
\mathbf{w}_i=[\omega_{1,i},\omega_{2,i},\ldots,\omega_{K,i}]. 
$$

Wird mit $\Gamma$ das zu erkennende Bild und mit $\Phi=\Gamma - \overline{\Gamma}$ die um den Mittelwert der Trainingsbilder subtrahierte Version des Bildes bezeichnet, dann sind
$$
\omega_{k}=\mathbf{u}_k^T \Phi
$$

die Koordinaten der Projektion von $\Phi$ in den Eigenspace und der dieses Bild repräsentierende Punkt
$$
\mathbf{w}=[\omega_{1},\omega_{2},\ldots,\omega_{K}].
$$

Das zu erkennende Bild wird dann dem Trainingsbild $\Phi_j$ zugeordnet, für welches gilt:
$$
j=argmin_{i} \left\{ d(\mathbf{w},\mathbf{w}_i) \right\}
$$

wobei mit $d(\mathbf{w},\mathbf{w}_i)$ die euklidische Distanz zwischen den Projektionen von $\Phi$ und $\Phi_i$ bezeichnet wird.

__Optional:__ Falls $\Phi_i$ nicht das einzige Bild einer Person in der Trainingsmenge ist, sondern für die entsprechende Person mehrere Trainingsbilder vorliegen, wird in der Distanzberechnung nicht $\Phi_i$, sondern der Mittelwert über alle zu dieser Person gehörenden Bilder eingesetzt:

$$
\overline{\Phi}=\frac{1}{|W|}\sum_{w \in W}^{}{\Phi_w} .
$$

Dabei bezeichnet $W$ die Menge aller der Indizees $w$, für die die $\Phi_w$ zur gleichen Person gehören. Im Praktikumsversuch muss diese Option nicht implementiert werden. Die im folgenden Abschnitt beschriebene Versuchsdurchführung bezieht sich auf den Fall, dass nur die Distanz zu Einzelbildern berechnet wird und das nächstliegende Bild ausgegeben wird.

Für die Mindestdistanz
$$
\epsilon =\min_{i} \left\{ d(\Phi,\Phi_i) \right\}
$$

wird in der Regel eine Schwelle $T$ definiert. Wenn $\epsilon > T$ ist, also eine relativ große Distanz zwischen dem zu erkennenden Bild und dem nächstliegenden Bild aus der Trainingsmenge besteht, wird davon ausgegangen, dass es sich um ein unbekanntes Gesicht handelt. Optional könnte dieses unbekannte Gesicht in die Trainingsmenge aufgenommen werden.

Schließlich muss noch der Fall behandelt werden, dass das eingelesene Bild kein Gesicht darstellt. Aufgrund der starken Projektion vom ursprünglichen Bildraum in den Eigenspace kann dieser Fall nicht durch eine Schwelle auf den Fehler $\epsilon$ erkannt werden. Es kann durchaus sein, dass ein Nicht-Gesichtsbild in die Umgebung eines Gesichtsbild im Eigenspace projiziert wird. Ein Nicht-Gesichtsbild wird aber eine relativ große Distanz $d(\Phi,\Phi_f)$ zwischen

$$
\Phi=\Gamma - \overline{\Gamma}
$$
und der Repräsentation im Eigenspace
$$
\Phi_f=\sum_{k=1}^{K}{\omega_k}\mathbf{u}_k
$$
aufweisen. Durch die Definition einer weiteren Schwelle $S$ auf $d(\Phi,\Phi_f)$ kann also erkannt werden, ob es sich überhaupt um ein Gesicht handelt. Im Versuch ist davon auszugehen, dass nur Gesichtsbilder verwendet werden, d.h. es muss nur der Test gegen die Schwelle $\epsilon$ implementiert werden.


 ## Vor dem Versuch zu klärende Fragen
 
 * Was sind Eigenvektoren und Eigenwerte?

** Antwort zu *"Was sind Eigenvektoren und Eigenwerte?"*: **  
* Eigenvektoren:  
Ein Eigenvektor $\overrightarrow{x}$ einer Matrix *M* ist ein vom Nullvektor verschiedener Vektor, dessen Richtung durch Multiplikation mit der Matrix nicht verändert wird. Ein Eigenvektor wird also nur gestreckt (oder gestaucht). Die Matrixmultiplikation von *M* mit $\overrightarrow{x}$ liefert das gleiche Produkt wie die Multiplikation des Eigenvektors mit dem Eigenwert λ. Die Eigenvektoren sind für eine quadratische Matrix immer reelwertig und orthogonal.  


* Eigenwerte:  
Der Streckungsfaktor λ heißt Eigenwert der Matrix. Die Eigenwerte sind die Faktoren für die zu der Matrix *M* ein Eigenvektor existiert.  

* Was versteht man unter Eigenfaces?

** Anwort zu *"Was versteht man unter Eigenfaces?"*: **  

Eigenfaces (Eigengesichter) sind das Resultat eines Verfahrens zur automatisierten Gesichtserkennung. Als Methode des maschinellen Sehens ermöglichen Eigengesichter die Erkennung von einer vorab trainierten Menge menschlicher Identitäten in Echtzeit - verlangen jedoch ein sehr hohes Maß an Homogenität in Bezug auf Lichtverhältnisse, Größe der Bilder, Rotation und Skalierung des Gesichts. Die *K* ausgewählten Eigenvektoren spannen einen *K*−dimensionalen Raum, den sogenannten Eigenspace auf. Die *K* Vektoren repräsentieren die *K* Merkmale, hinsichtlich derer die Bilder der Trainingsdatenmenge am stärksten variieren (also die Eigenvektoren, die die höchsten Eigenwerte haben). Die Eigenfaces stehen orthogonal zueinander und bilden eine Basis des K-dimensionalen Raumes. Für eine Transformation in diesen Raum lassen sich die Originaldaten dann als Linearkombination der Eigenfaces darstellen, und dabei dann in eine komprimierte Form bringen, um somit eine Reduktion der Komplexität zu erreichen.

* Die PCA ist u.a. im entsprechenden Kapitel des Dokuments [J. Maucher; Feature Selection and Extraction](https://docs.google.com/document/d/13cc9RGeIvsV5JsC-MsymJ6aaJs_ZsUKEwqaOOO8IL8Q/edit?usp=sharing)) beschrieben. Wie kann mit der PCA eine Dimensionalitätsreduktion durchgeführt werden?

** Antwort zu *"Wie kann mit der PCA eine Dimensionalitätsreduktion durchgeführt werden?"*: **  

Ein Bild wird in einen Block von $x_{1}$ x $x_{2}$ Pixel eingeteilt. Dadurch entsteht der (aus den $x_{1}$ x $x_{2}$ Pixeln) *n*-dimensionale Originalraum (Ortsbereich). Im transformierten Raum (Frequenzbereich) erhält man typischerweise hohe Werte im Bereich niedriger Frequenzen und niedrige Werte im Bereich hoher Frequenzen. Man sagt auch, dass die Information eines Blockes nahezu ausschließlich in den niedrigen Frequenzen liegt. Die PCA (Hauptkomponentenanalyse) bildet den Originalraum in einen transformierten Raum ab, in welchem die Varianz des Datensatzes ungleich über die neuen Dimensionen verteilt ist. Dazu sollen nur noch diejenigen Komponenten berücksichtigt werden, welche die meisten Informationen tragen (und wenig aussagekräftige Komponenten sollen dabei ignoriert werden). Um die Vektoren des Ursprungsraums für eine Dimensionalitätsreduktion in den reduzierten zu übertragen, wird eine Transformationsmatrix benötigt, in der die Spaltenvektoren paarweise orthogonal zueinander sind, sodass sie einen neuen Raum aufspannen können. Dazu wird die Kovarianzmatrix des Ursprungsraum gebildet und ihre Eigenvektoren berechnet (die Matrix ist dabei quadratisch, die Eigenvektoren sind zueinander orthogonal). Die Eigenvektoren spannen dabei dann den reduzierten Raum auf und stellen zusammen eine Basis des reduzierten Raumes dar (jeder Vektor des reduzierten Raumes lässt sich als Linearkombination der Eigenvektoren darstellen). Wie bereits erwähnt gilt für Eigenvektoren: je höher der Eigenwert des Vektors, desto höher die Varianz entlang der Achse, die durch den Eigenvektor repräsentiert wird. Bei Nutzung aller Eigenvektoren, findet bei der Transformation lediglich eine Drehung des Raumes statt. Wählt man stattdessen nur diejenigen Eigenvektoren aus, bei denen die Eigenwerte am höchsten sind, dann findet bei der Transformation nur ein geringer Informationsverlust statt (da nur die Achsen mit der größten Streuung ausgewählt wurden) bei gleichzeitiger Dimensionalitätsreduktion. Diese ausgewählten Eigenvektoren sind dabei dann die Eigenfaces.  

Durchführung der Transformation:

* *Normierung*:  
Die Datenmenge der Matrix *X* wird zunächst in eine mittelwertfreie Matrix *X'* überführt. Hierzu wird für jedes Merkmal $x_{j}$ der Mittelwert über alle *N* Datensätze berechnet, anschliessend werden alle Werte normiert.  


* *Berechnung der Kovarianzmatrix*:  
Anschliessend wird die Varianz des Merkmals $x_{j}$ sowie die Kovarianz zwischen den Merkmalen $x_{j}$ und $x_{k}$  berechnet und in die Kovarianzmatrix *C* eingetragen. Die Kovarianzmatrix ist eine symmetrische und quadratische Matrix, ihre Zeilen- und Spaltenzahl ist durch die Anzahl der Merkmale *d* gegeben. Die Varianzen $c_{jj}$ auf der Hauptdiagonale beschreiben, wie stark die Werte des Attributs $x_{j}$ um ihren Mittelwert streuen. Die Kovarianzwerte $c_{jk}$ beschreiben die Korrelation zwischen den Attributen $x_{j}$ und $x_{k}$. Ein Kovarianzwert von $c_{jk}$ = 0 bedeutet, dass keine Korrelation zwischen den Merkmalen $x_{j}$ und $x_{k}$ besteht. Sind in einer Kovarianzmatrix alle Nicht-Hauptdiagonalelemente = 0, dann existieren keine Korrelationen zwischen den Merkmalen.


* *Eigenvektoren und Eigenwerte der Kovarianzmatrix bestimmen*:  
Nun werden die Eigenvektoren und Eigenwerte der Kovarianzmatrix berechnet. Da *C* quadratisch und symmetrisch ist, müssen die Eigenvektoren reelwertig und orthogonal zueinander sein.  


* *Ordnen der Eigenvektoren und Festlegung der Anzahl der Dimensionen im transformierten Raum*:  
Eigenvektoren mit höherem Eigenwert sind bedeutungsvoller (bzw. die durch sie gegebene Richtung enthält mehr Information) als Eigenvektoren mit niedrigem Eigenwert. Für eine Reduktion der Dimensionen (Richtungen), welchen der geringste Informationsgehalt inne liegt, erweist es sich als effizient die Eigenvektoren als Spalten in eine Matrix *F* derart anzuordnen, dass der Eigenwert der Eigenvektoren von links nach rechts abnimmt. D.h. der Eigenvektor mit dem höchsten Eigenwert ist die erste Spalte, der Eigenvektor mit dem kleinsten Eigenwert die letzte Spalte der Matrix *F*. Nach dieser Anordnung muß nun entschieden werden, wie viele Dimensionen im transformierten Raum enthalten sein sollen. Die Anzahl *w* der Dimensionen muß im Bereich *1* $\le$ *w* $\le$ *d* liegen wobei d die Anzahl der Dimensionen (Merkmale) im Ausgangsraum angibt. Abhängig vom jeweiligen Problem kann w fest vorgegeben sein oder aber w wird dynamisch nach der Ermittlung der Eigenwerte festgelegt. Hierbei kann z.B. eine relative Schwelle für die Eigenwerte definiert werden und alle d-w Eigenvektoren, deren Eigenwert unter dieser Schwelle liegt gestrichen werden. Nach der Festlegung von w, werden aus Matrix F die letzten w Spalten gestrichen.  


* *Daten transformieren*:  
Nach der Festlegung der Dimensionsanzahl *w* im transformierten Raum und der entsprechenden Elimination der *d-w* letzten Spalten der Matrix *F* enthält *F* genau *w* Spalten und *d* Zeilen. Die Datenmenge *X* enthält *N* Datensätze, die jeweils durch *d* Merkmale beschrieben werden. Für die Transformation wird die transponierte normierte Datenmenge $X'^T$ mit der Transponierten der Matrix *F* multipliziert. In der Matrixmultiplikation auf der rechten Seite dieser Gleichung werden die Eigenvektoren (Zeilen von $F^T$) mit den einzelnen normierten Datensätzen (Spalten von $X'^T$) Skalar-multipliziert, was der Projektion der Datenpunkte auf den jeweiligen Eigenvektor gleichkommt. In der ersten Zeile der Matrix *Y* werden sich also die Projektionen der *N* Datenpunkte auf die Hauptachse befinden, in der zweiten Zeile die Projektionen der Datenpunkte auf $z_{C,2}$ usw. 




* Wie werden mit dem Python Modul \emph{Image} Bilder in ein Python-Programm geladen?

** Antwort zu *"Wie werden mit dem Python Modul '\emph{}' Bilder in ein Python-Programm geladen?"*: **  

Beispielhafter Code: 

*//Modul importieren*  
from PIL import Image  

*//Pfad zum Bild*  
im = Image.open("../example.jpg")  

*//Bild laden*  
pic = im.load()


# Versuchsdurchführung

## Einlesen der Gesichtsbilder in Numpy Arrays
Laden Sie vom Skripteserver das Verzeichnis _Gesichtsbilder_ auf Ihren Rechner. Darin enthalten sind

* das Unterverzeichnis _training_, welches je 3 Bilder jedes Studenten enthält,
* das Unterverzeichnis _test_, welches die nicht in _training_ enthaltenen Bilder enthält. Ein Bild von jedem Studenten.

Mit der unten gegebenen Funktion _parseDirectory(directoryName,extension)_ wird eine Liste aller Dateinamen des Typs _extension_ im Verzeichnis _directoryName_ angelegt.

__Aufgabe:__

1. Legen Sie mit dieser Funktion eine Liste mit allen Dateinamen des Typs _extension='.png'_ im Verzeichnis _training_ (enthält die Trainingsbilder) an.
2. Implementieren Sie eine Funktion _readImageToNumpyData(imageList)_, der eine Liste aller Dateinamen der Trainingsbilder übergeben wird. Die Funktion gibt ein Numpy-Array zurück. Jede Zeile dieses Arrays enthält ein _.png_-Bild in serialisierter Form. Hierzu ist jedes einzelne Bild zunächst mit der Funktion [_matplotlib.image.imread(filename)_](http://matplotlib.org/1.3.0/users/image_tutorial.html) in ein zweidimensionales Numpy Array _img_ zu lesen. Durch den Aufruf von _img.shape=(1,-1)_ wird das zweidimensionale Numpy Array zu einem eindimensionalen Array serialisiert. Danach muss eine Normierung aller Werte in den Bereich zwischen 0 und 1 durchgeführt werden. Hierzu müssen alle Pixelwerte eines Bildes durch den im jeweiligen Bild vorkommenden Maximalwert geteilt werden.  

In [23]:
%matplotlib inline
from os.path import isdir,join,normpath
from os import listdir
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as mplimg
import os.path



In [24]:
def parseDirectory(directoryName,extension):
    '''This method returns a list of all filenames in the Directory directoryName. 
    For each file the complete absolute path is given in a normalized manner (with 
    double backslashes). Moreover only files with the specified extension are returned in 
    the list.
    '''
    if not isdir(directoryName): return
    imagefilenameslist=sorted([
        normpath(join(directoryName, fname))
        for fname in listdir(directoryName)
        if fname.lower().endswith('.'+extension)            
        ])
    return imagefilenameslist

In [26]:
training_images = os.path.dirname(os.getcwd()) + "/Resources/Gesichtsbilder/training/"
test_images = os.path.dirname(os.getcwd()) + "/Resources/Gesichtsbilder/test/"

training_image_list = parseDirectory(training_images, 'png')
test_image_list = parseDirectory(test_images, 'png')


In [8]:
def readImageToNumpyData(imageList):
    images_list = []
    for image in imageList:
        img = img=mplimg.imread(image)
        pixeldata = np.array(img)
        pixeldata = pixeldata.flatten()
        pixeldata = pixeldata / max(pixeldata)
        images_list.append(pixeldata)
    return np.array(images_list)

In [ ]:
training_image_list = readImageToNumpyData(training_image_list)

## Berechnung des Durchschnittbildes

__Aufgaben:__

1. Die von der Funktion _convertImgListToNumpyData(imgList)_ zurückgelieferte Matrix enthält in ihren Zeilen alle Trainingsbilder. Aus diesen Trainingsbildern ist nach der Gleichung für [$\overline{\Gamma}$](#theoryEig) das Durchschnittsbild zu berechnen, z.B. durch Anwendung der Numpy-Funktion _average_. Das Durchschnittsbild ist von allen Bildern abzuziehen (Gleichung [$\Phi_i$](#theoryEig)). Das daraus resultierende Numpy-Array enthält die mittelwertfreien Repräsentationen der Trainingsbilder und wird im Folgenden mit _NormedArrayOfFaces_ bezeichnet.

2. Zeigen Sie das Durchschnittsbild mithilfe der [matplotlib.pyplot_-Funktion _imshow()](http://matplotlib.org/1.3.0/users/image_tutorial.html) an. Hierzu muss das eindimensionale Numpy Array, welches das Durchschnittsbild enthält, in ein zweidimensionales Array der ursprünglichen Bildgröße umgewandelt werden (Numpy Funktionen _shape()_ oder _reshape()_)  

__Wichtiger Hinweis:__ Das Numpy-Array __NormedArrayOfFaces__ ist die Transpornierte $X^T$ der Matrix $X$ aus Gleichung [$X$](#theoryEig).

In [9]:
 def get_average_image(images):
    average = range(len(images[0]))
    for idx in np.arange(len(images[0])):
        array = [x[idx] for x in images]
        average[idx] = np.average(array)
    return np.array(average)
    
def get_normed_array_of_faces(images, average_image):
    normed = []
    for idx in np.arange(len(images)):
        normed.append(images[idx] - average_image)
    return np.array(normed)

def reshape_image(image, width=150, height=220):
    shaped_image = image.copy()
    shaped_image.shape = (height, width)
    return shaped_image

In [ ]:
average_image = get_average_image(training_image_list)
normed_array_of_faces = get_normed_array_of_faces(training_image_list, average_image)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(reshape_image(average_image), cmap='gray')
plt.colorbar()
imgplot = plt.title("Durchschnittsbild", size=15)

## Berechnung der Eigenfaces

__Aufgaben:__

1. Implementieren Sie die Funktion _calculateEigenfaces(adjfaces,width,height)_. Dieser Funktion werden die normierten Bilder _NormedArrayOfFaces_ zusammen mit der Bildbreite und -höhe übergeben. Zurück liefert die Funktion ein Numpy-Array, dessen Zeilen die berechneten normierten Eigenfaces sind. Die Berechnung der Eigenfaces ist im Theorieteil Abschnitt [Bestimmung der Eigenfaces](#theoryEig) beschrieben. Für die Python-Implementierung können Sie folgende Hinweise berücksichtigen:
    * Berechnung der transponierten eines Numpy-Arrays $A$ mit der Numpy-Methode _transpose()_
    * Matrixmultiplikation zweier Numpy-Arrays $A$ und $B$ mit der Numpy-Funktion _dot()_
    * Berechnung der Eigenvektoren und Eigenvalues eines Numpy Arrays $A$ mit der Numpy-Funktion _linalg.eigh()_
    * Sortierung von Numpy-Arrays mit den Numpy-Funktionen _sort()_ und _argsort()_.
2. Aus dem von der Funktion _calculateEigenfaces(adjfaces,width,height)_ zurück gelieferten Array von Eigenfaces sind die $K$ relevantesten auszuwählen. Dieses reduzierte Array wird im Folgenden mit _Usub_ bezeichnet. Im Versuch kann $K=6$ eingestellt werden.
3. Zeigen Sie die $K=6$ wichtigsten Eigenfaces als Bilder mit der [matplotlib.pyplot_-Funktion _imshow()](http://matplotlib.org/1.3.0/users/image_tutorial.html) an.

In [10]:
def calculateEigenfaces(adjfaces, width = 150, height = 220):
    eigenvecs_of_CV = []

    reduced_matrix = np.dot(adjfaces, np.transpose(adjfaces))
    M = reduced_matrix.shape[0]
   
    (eigenval_of_reduced, eigenvecs_of_reduced) = np.linalg.eigh(reduced_matrix)
    print "M= " + str(M)
    print "adjfaces= " + str(adjfaces.shape)
    
    
    for i in range(M):
        u = [0] * adjfaces.shape[1]
        for k in range(M):
            
            u = u + (eigenvecs_of_reduced[i, k] * adjfaces[k])
        eigenvecs_of_CV.append(u)

    return np.array([eigenvec for _,eigenvec in sorted(zip(eigenval_of_reduced,eigenvecs_of_CV))])

In [11]:
eigenfaces = calculateEigenfaces(normed_array_of_faces, )
usub = eigenfaces[:6]

In [12]:
M= 63
adjfaces= (63, 33000)

**!!! -> Zu Jannik's Notebook: !!!**  

** Ergebnis zu *"Berechnung der Eigenfaces"* mit Unterpunkt *"Zeigen Sie die  K=6
  wichtigsten Eigenfaces als Bilder an"* **  
  
Trotz Dimensionsreduktion sind die Eigenfaces noch einigermaßen gut als menschliche Gesichter zu erkennen. Dabei legt jedoch jedes der K=6 Eigenfaces bestimmte Schwerpunkte im Bezug auf die Merkmale (teilweise überschneiden sich diese natürlich auch, allerdings in unterschiedlicher Ausprägung). Bilder 2 und 6 fokussieren sich beide auf eine Brille sowie Haare, Bild 6 aber ausserdem auch besonders auf Bart und Kinnpartie. Bild 5 hingegen fokussiert sich überwiegend auf die Stirnpartie mit Haaren. Bei Bild 5 wird dabei bspw. ein Pony charakterisiert, während bei Bild 3 eine hohe Stirn mit entsprechend hohem Haaransatz charakterisiert wird.

## Transformation der normierten Trainingsbilder in den Eigenface Raum

__Aufgabe:__

Die im vorigen Schritt angelegten $K$ relevantesten Eigenfaces spannen den sogenannten _Eigenface-Raum_ auf. Für jedes der normierten Trainingsbilder, also für jede Zeile aus _NormedArrayOfFaces_, sind die Koordinaten im Eigenface-Raum entsprechend der Gleichung für [$\omega_{k,i}$](#theoryRec) definierten Transformation zu berechnen.

In [13]:
import sys
def calculateOmegaValues(normedArrayOfFaces, eigenspace, verbose=False):
    #Verify
    if (eigenspace.shape[1] == normedArrayOfFaces.shape[1]):
        if (verbose):
            print 'Same Dimensions'
    else:
        sys.stderr.write('The Matrices do not have the same dimensions')
    
    #Logic
    omega_value_list = np.zeros((normedArrayOfFaces.shape[0], eigenspace.shape[0]))
    for i,image in enumerate(normedArrayOfFaces):
        for k,es in enumerate(eigenspace):
            #Both vectors are supplied horizontal
            #dot product expects h*v to calculate a single value
            omega_i_k = np.dot(es, np.transpose(image))
            omega_value_list[i, k] = omega_i_k
            
    return omega_value_list 

## Erkennung

__Aufgaben:__

1. Wählen Sie ein Bild aus dem Verzeichnis _test_ aus. Das ausgewählte zu erkennende Bild ist als Numpy-Array darzustellen. Eine Normierung der Pixelwerte in den Bereich zwischen 0 und 1 ist durchzuführen (wie bereits oben beschrieben). Schließlich muss auch von diesem Bild das Durchschnittsbild aller Trainingsbilder abgezogen werden. Diese Prozessschritte entsprechen der oben beschriebenen Vorverarbeitung der Trainingsbilder. Das resultierende normierte und mittelwertfreie Bild wird im Folgenden mit _NormedTestFace_ bezeichnet.
3. Danach sind die Koordinaten des _NormedTestFace_ im Eigenface-Raum nach Gleichung [$\omega_{k}$](#theoryRec) zu berechnen und das in diesem Raum nächstliegende Trainingsbild zu bestimmen.

In [14]:
#Your Code

__Aufgaben:__
1. Führen Sie die implementierte Gesichtserkennung für alle Bilder im Verzeichnis _test_ aus. Zeigen Sie jeweils das Testbild, das zugehörige erkannte Bild und die Distanz zwischen beiden Bildern an.
2. Bestimmen Sie für die Werte $K=5,K=10$ und $K=15$ ($K$ ist die Anzahl der verwendeten Eigenfaces) die Rate falsch erkannter Bilder. 

**!!! -> Zu Jannik's Notebook: !!!**  

** Ergebnis zu *"Erkennung"* mit Unterpunkt *"Bestimmen Sie für die Werte K=5, K=10 und K=15 die Rate falsch erkannter Bilder"* **  


  

Der Versuch wurde mit K=5, K=10, K=15 und K=20 Eigenfaces durchgeführt. Während bei K=5 und K=10 4 von 21 Personen falsch erkannt wurden (ca. 19% Fehlerrate), waren es bei K=15 nur noch 3 von 21 Personen (ca. 14% Fehlerrate) und bei K=20 sogar nur noch 2 von 21 Personen (ca. 9,5% Fehlerrate). Hieraus würde sich ein Zusammenhang ableiten lassen, der sich wie folgt begründen würde: Je mehr Eigenfaces verwendet werden (also je höher die Anzahl der Dimensionen ist), desto besser wird die Fehlerrate bei der Erkennung. Es scheint also, dass mehr Dimensionen gleichbedeutend mit einer genaueren Darstellung der einzelnen Merkmale sind, da auch mehr Eigenfaces zur Verfügung stehen, die für einen Linearkombination zur Abbildung in den reduzierten Raum nutzbar sind.  
Interessanterweise ist zu beobachten, das bei K=20 zwar Personen richtig erkannt werden, die auf einem Bild eine Brille, auf dem anderen aber keine tragen, wiederrum als extremstes Beispiel eines Fehlers eine Frau und ein Mann als gleich zugeordnet werden, obwohl die Unterschiede sehr offensichtlich sind: Frau/Mann, ohne Bart/mit Bart, Pony/hohe Stirn, offene Aufgen/zugekniffene Augen... - die einzigen Gemeinsamkeiten würden sich noch weitläufig im dunklen Oberteil und der rechten Seite der Frisur  finden. Ansonsten sind beide völlig unterschiedlich. Bei dem anderen hier falsch erkannten Bild ist es schon offensichtlicher, warum diese als gleich/ähnlich erkannt wurden: selbe Neiung des Kopfes, ähnliche Frisur, Augenbrauen, Augen, Mund, Bart - der einzige wirklich eindeutige Unterschied ist die Farbe der Oberteile (hell/dunkel).  
Bei K=5 und K=10 scheint der Hauptaugenmerk bei der Unterscheidung sehr darauf zu liegen, wo der Kopf einer Person in einem Bild liegt (also in welchem Bereich). Wenn unterschiedliche Personen als gleich "erkannt" wurden, dann sind beide Köpfe in der Regel im gleichen Bereich des Bildauschnitts zu finden und/oder haben die gleiche Neigung. Auch die Art der Frisur (Pony oder hoher Haaransatz) scheint ein stärkeres Kriterium zu sein.
